1. Базовый вариант (25 баллов) 

Здесь я предлагаю вам сделать API для GitHub. У гитхаба есть публичный API, но для учебных целей мы сделаем велосипед.

Реализуйте следующие функции:
-get_user_info(username) - возвращает словарь с информацией о юзере. Ключи: имя (которое НЕ никнейм), организация, местоположение, число подписчиков, число репозиториев.

-get_user_repositories(username) - возвращает список публичных репозиториев пользователя в виде списка словарей. Ключи: юзер, имя репозитория, язык.

-list_repository_contents(username, repository, repository_path) - возвращает список файлов и папок в репозитории repository пользователя username по пути repository_path. По сути это аналогично использованию "ls repository_path" локально, мы просто выводим список файлов по указанному пути.

-download_file(username, repository, remote_file_path, local_file_path) - скачивание файла, который имеет путь remote_file_path в репозитории и сохранение локально по пути local_file_path. При попытке скачать папку можно бросить исключение или же скачать всю папку.


2. Вариант посложнее (25 баллов + 15 доп) 

Предлагаю использовать вот этот сервис http://hollywood.mit.edu/GENSCAN.html
Он способен находить и вырезать интроны в переданной нуклеотидной последовательности. Делает он это не очень хорошо, но это лучше, чем ничего. К тому же у него действительно нет публичного API.


Реализуйте следующую функцию:

-run_genscan(sequence=None, sequence_file=None, organism="Vertebrate", exon_cutoff=1.00, sequence_name="") - выполняет запрос аналогичный заполнению формы на сайте. Принимает на вход все параметры, которые можно указать на сайте (кроме Print options).

-sequence - последовательность в виде строки или любого удобного вам типа, sequence_file - путь к файлу с последовательностью, который будет загружен и использован вместо sequence. Функция должна будет возвращать объект типа GenscanOutput. Про него дальше.

Реализуйте класс GenscanOutput у него должны быть следующие атрибуты:

-status - статус запроса

-cds_list - список предсказанных белковых последовательностей с учётом сплайсинга (в самом конце результатов с сайта)

-intron_list - список найденных интронов. Один интрон можно представить любым типом данных, но он должен хранить информацию о его порядковом номере, его начале и конце. Информациб о интронах можно получить из первой таблицы в результатах на сайте.

-exon_list - всё аналогично интронам, но только с экзонами.
По желанию ещё любые данные, которые вы найдёте в результатах

In [1]:
import requests
from bs4 import BeautifulSoup

## Сделаем базовый вариант - API для GitHub

Реализуйте следующие функции:

1) get_user_info(username) - возвращает словарь с информацией о юзере. Ключи: имя (которое НЕ никнейм), организация, местоположение, число подписчиков, число репозиториев.

2) get_user_repositories(username) - возвращает список публичных репозиториев пользователя в виде списка словарей. Ключи: юзер, имя репозитория, язык.

3) list_repository_contents(username, repository, repository_path) - возвращает список файлов и папок в репозитории repository пользователя username по пути repository_path. По сути это аналогично использованию "ls repository_path" локально, мы просто выводим список файлов по указанному пути.

4) download_file(username, repository, remote_file_path, local_file_path) - скачивание файла, который имеет путь remote_file_path в репозитории и сохранение локально по пути local_file_path. При попытке скачать папку можно бросить исключение или же скачать всю папку.

In [35]:
url = 'https://github.com/krglkvrmn' # можно сюда {} вот так вставить имя

In [59]:
url = 'https://github.com/immbiochem'

In [36]:
resp = requests.get(url)

In [37]:
soup = BeautifulSoup(resp.content)

### Первая функция

In [115]:
def get_user_info(username):
    url = 'https://github.com/' + username
    user_dict = {'Name':'Nothing', 
                 'Organization':'Nothing',
                 'Number of repositories':'Nothing',
                 'Number of followers':'Nothing',
                'Location' : 'Nothing'}
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content)
    action_dict = {'Name': (lambda soup: soup.find('span', itemprop = 'name').text.strip()), 
                 'Organization':(lambda soup: soup.find('span', class_ = "p-org").text),
                 'Number of repositories':(lambda soup: soup.find_all('span', class_ = 'Counter')[0].text),
                 'Number of followers':(lambda soup: soup.find('span', class_ = "text-bold color-fg-default").text),
                'Location' : (lambda soup: soup.find('span', class_ = "p-label").text)}
    for name in user_dict:
        try:
            user_dict[name] = action_dict[name](soup)
        except:
            pass
    return user_dict
        

#### Демонстрация

In [116]:
slovar = get_user_info('krglkvrmn')
print(slovar)

{'Name': 'Roman Kruglikov', 'Organization': 'Nothing', 'Number of repositories': '11', 'Number of followers': '1', 'Location': 'Moscow'}


In [117]:
slovar = get_user_info('immbiochem')
print(slovar)

{'Name': 'Semyon Kupriyanov', 'Organization': 'SUSMU', 'Number of repositories': '4', 'Number of followers': 'Nothing', 'Location': 'Nothing'}


### Вторая функция

In [17]:
import re

In [111]:
def get_user_repositories(username):
    url = 'https://github.com/'+username+'?tab=repositories'
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content)
    rep = soup.find_all('li', class_=re.compile('col-+'))
    reps_list = []
    for i in range(len(rep)):
        if rep[i].find('span', class_=re.compile('Label')).text == 'Public':
            block = {'User': soup.find('span', itemprop='name').text.strip()}
            try:
                block['Name'] = rep[i].find('a', itemprop="name codeRepository").text.strip()
            except:
                block['Name'] = 'None'
            try:
                block['Language'] = rep[i].find(['ul', 'span'], itemprop=re.compile('pro+')).text
            except:
                block['Language'] = 'None'
            reps_list.append(block)
    return reps_list

#### Демонстрация

In [113]:
print(get_user_repositories('krglkvrmn'))

[{'User': 'Roman Kruglikov', 'Name': 'FlowSOM', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'Clipboard_translater', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'Virtual_environment_research', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'BI_2021_Python', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'IB_Music_project', 'Language': 'Jupyter Notebook'}, {'User': 'Roman Kruglikov', 'Name': 'Asteroids_pygame_tutorial', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'BI_2021_spring_project', 'Language': 'Jupyter Notebook'}, {'User': 'Roman Kruglikov', 'Name': 'SeqTools', 'Language': 'Python'}, {'User': 'Roman Kruglikov', 'Name': 'BI_Statistics_projects', 'Language': 'None'}, {'User': 'Roman Kruglikov', 'Name': 'BI_Python', 'Language': 'Jupyter Notebook'}, {'User': 'Roman Kruglikov', 'Name': 'MAG_gi_plasmid_analysis', 'Language': 'Jupyter Notebook'}]


In [114]:
print(get_user_repositories('immbiochem'))

[{'User': 'Semyon Kupriyanov', 'Name': 'BI_ML_2021', 'Language': 'None'}, {'User': 'Semyon Kupriyanov', 'Name': 'BI_2021_Python', 'Language': 'Jupyter Notebook'}, {'User': 'Semyon Kupriyanov', 'Name': 'BI_Stat_2021', 'Language': 'HTML'}, {'User': 'Semyon Kupriyanov', 'Name': 'bf_course', 'Language': 'Jupyter Notebook'}]


### Третья функция